# Locality-sensitive hashing for detecting coreferences

Problem: We want to find mentions that cold refer to the same entity. Currently we compare each mention with each other, and if mention 1 contains mention 0 as a word, then we consider mention 0 and mention 1 potential coreferences. This approach does not scale well with the number of mentions.

Solution: try out how to use locality-sensitive hashing to reduce the number of comparisons to make. While there is optimized software available to do this, I think that a good start is a solution without external dependencies: no need to check compatibility with other requirements, and data are already preprocessed which should make the task computationally simple. 

How does LSH work?
1. Shingling: convert text to sparse vectors. I will start with shingling at the word level and think about alternatives later.
    - One-hot encoding
    - Define vocabulary of all shingles
2. MinHashing: create signatures based on randomly reordering the vocabulary and recording the first shingle that is in mention $i$.
3. Cut the signature into bands and assign--using a hash function--all mentions to buckets. 
    - More bands means larger buckets
    - Need to use the same function for the same band $j$ for all mentions. Can use different functions for different bands, but not required. 

## Speeding up the hashing
The approach with min hashing is not feasible for high-dimensional data sets because it is expensive to randomly shuffle large arrays and/or store large binary arrays in memory. But there are alternatives. One is with random projections, as discussed in the wikipedia page, in [this video](https://www.youtube.com/watch?v=Arni-zkqMBA) and in [this book](http://infolab.stanford.edu/~ullman/mmds/ch3n.pdf) (section 3.7.2 Random Hyperplanes and the Cosine Distance).

What is the idea?
- Use the cosine similarity between two binary vectors with the cosine distance
- Suppose we pick a hyperplane through the origin. We do so by choosing a vector $v$ that is perpendicular to the hyperplane. 
    - Then, either two vectors $x$ and $y$ lie on the same side of the hyperplane or they do not. 
- Then, choose a random vector $v_f$. 
    - we can build a hashing function $f$ that assigns the same value to $x$ and $y$ if they lie on the same side of the hyperplane that $v$ is perpendicular to (the dot product $x.v$ and $y.v$ will be informative about this.)
    - (the angle $\theta$ between $x$ and $y$ will determine the probability that $x$ and $y$ are on the same side of a given hyperplane; see book for intuition)
    - the family $F$ of functions defined by vectors $v_f$ is locality-sensitive for the cosine distance. And this is very similar to the Jaccard-distance family, up to some scaling factor. 
- We can then build sketches (see the code [here](https://github.com/brandonrobertz/SparseLSH/blob/11f381560a94c8d74af55b3db5e8db1bbddfc212/sparselsh/lsh.py#L140)) by using random vector whose elements are either +1 or -1.
    - consider a random vector $v$ where $v_i \in \{-1, 1\} \forall i$.
    - we calculate the dot product for a random vector $v$ and vector $x$.
    - the dot product is the difference between the sum of all elements $x[i]$ for $i: v[i] = 1$, and the sum of all elements $x[i]$ for $i: v[i] = -1$. 
    - We repeat this for multiple vectors $v$ and store whether the dot product is positive or negative (again by $+1$ or $-1$). Since a dot product of $0$ is unlikely, we can handle such cases arbitrarily. 
    - The result of this is called a **sketch** (which is the same as a signature, see [p. 49 here](https://web.stanford.edu/class/cs246/slides/04-lsh_theory.pdf))
- I do not understand the example 3.22: does it imply that we have to take a large number of random vectors? what is the 
- p. 99: cosine distance makes sense in spaces [...] where points are vectors with integer components or Boolean components -- thus, we can use it here. 

Now I am not sure how SparseLSH implements the whole thing. They have different distance options, but the hashing is, as far as I understand, the same for all options. What is the theory behind this? Can I just use this hashing instead of my hashing, and then continue with the signature as before (which is essentially the Jaccard distance??)
    - or is this perhaps what is discussed [here](https://ai.stackexchange.com/questions/37199/clustering-by-using-locality-sensitive-hashing-after-random-projection)?
    - in fact, the slides from Stanford seem to imply that using cosine distance for LSH in the same way min-hashing was used for the Jaccard distance.

The output from the random projections is again a (denser) vector of -1s and +1s. Because this carries much less information than the real-valued vectors from the minhashing, the banding technique does not work--too many items would have the same band. So, what does the SparseLSH apply then? What do they write in the book? What does wikipedia say?
- see [here, p. 58](https://web.stanford.edu/class/cs246/slides/04-lsh_theory.pdf): they apply the bands technique to the Euclidean distance

[continue in pdf]

section 3.6: family of locality-senstive functions
- minhash function is one family of locality-sensitive functions


Some lessons I learned
- It is important to use the binary vectors. If using the min-hashed vectors, longer words are closer to longer ones (and shorter to shorter)


In [1]:
from load_coreferences import load_coreferences
# import lsh 
import REL # install with pip install -e ../REL/. (or pip install -e ../REL/.[develop])
import copy

import numpy as np
from REL import lsh 


Next steps, 16/01/23
- [ ] tidy the lsh class
    - remove comments etc
    - fix TODOs
    - rename the class - it is not MinHash anymore
- [ ] rerun all tests. does profiling work now? why not?
- [ ] better understand time complexity
    - how does it depend on the parameters? -- the youtube video is helpful
    - automatically adjust the inputs to LSH as a function of the number of mentions?
- [ ] improve ROC curves
    - they look a bit funny, are they correct?
    - what does precision measure? it just means that we retrieve many more items in a group right? can we display the implied group size? this could be informative.
    - repeat the exercises for all mentions, no only coreferring mentions? 
- [ ] try to make faster
    - still quite some overhead of with_coref function (but lower than before)
    - is there a way to improve upon it?
    - profiling output:
        1. [x] encode_binary - try sklearn? but another dependency(?) 
        2. [ ] get_candidates. Now this is the bottleneck
        3. [ ] (idx_unique_multidim) -- but at first sight it seems unavoidable and already optimised
            - the function idx_unique_multidim itself is fast. but it is called for each band. can it be vectorized. 
        - also: are there better ways to choose the vectors (to make sure that we have many that are sufficiently different from each other?)
    - add external options for benchmarking -- benchmark for efficiency and effectiveness
        - faiss 
        - datasketch
- [ ] fix script with evaluations from Erik (see PR on github)
- [ ] scaling of mentions: use all data sets (current default is 50). go back to less scaling? 
    - problem with document_nstack "1208testb_300". why?
    - I am not sure anymore how good this stacking is for testing lsh: by stacking the same mentions, the number of comparisons mechanically increases. But this seems to be a problem only for lsh, ie, does it produce an lower bound for the efficiency gain of going from "all" to "lsh"?
- [X] check what is going on for msmarco: size of the candidate groups?
    - print this out at the end of the hashing for information
    - maybe store it also as another output? 
- [ ] inputs to class should be band length and number of bands; then signature can be calculated directly for any inputs
- [x] try out an option with even longer signature and longer bands? would that perhaps increase precision without lowering recall? 
- [ ] speed up grouping? avoid the quadratic time complexity there? -- check the sorted list approach
- [ ] scaling of mentions: set limit of number of mentions, for instance around as many as in 1208testb_100.
- [ ] strictly speaking the parameters for LSH should be chosen based on the training data set, but currently they are chosen with the test data set. 

0. Load data 

In [2]:
raw_mentions = load_coreferences(drop_duplicates=False)


In [3]:
len(raw_mentions)

632

In [4]:
mentions = {i: m for i, m in enumerate(raw_mentions)}
# stack them on top of each other 

mentions_scaled = copy.copy(mentions)

idx = len(mentions_scaled)
scaling_factor = 5
for i in range(1, scaling_factor):
    for idx_old in mentions.keys():
        m = mentions[idx_old]
        mentions_scaled[idx] = m 
        idx += 1

### How to integrate with REL?

In [6]:
# out = {i: {"shingles": lsh.k_shingle(m, 4)} for i, m in zip(range(len(mentions_rel), mentions)) }
# mentions_rel
mentions_rel = [
    'German', 'British', 'Brussels', 'European Commission', 'German',
    'British', 'Germany', 'European Union', 'Britain', 'Commission', 
    'European Union', 'Franz Fischler', 'Britain', 'France', 'BSE', 
    'Spanish', 'Loyola de Palacio', 'France', 'Britain', 'BSE', 'British', 'German', 
    'British', 'Europe', 'Germany', 'Bonn', 'British', 'Germany', 'Britain', 'British'
]

mylsh = lsh.LSHMinHash(mentions=mentions_rel, shingle_size=2, signature_size=900, band_length=15)
mylsh.cluster()
mylsh.summarise()



took 0.007684469223022461 seconds for 30 mentions
average, min, max cluster size: 6.07, 0, 13


How to deal with input where only one item is repeated? -- put as a test!

In [10]:
import lsh 
import numpy as np

test_mentions = ['EEC', 'EEC'] # this fails. ['German', 'German'] does not. neither does ['EEC', 'EEC', 'German']
test_mentions = ['EEC', 'ABC'] # this also fails. ['EEC', 'ABCde'] does not. 
# the problem is that when there is no mention of at least the inputted shingle size, the binary vectors cannot be calculated

mylsh = lsh.LSHMinHash(mentions=test_mentions, shingle_size=4, signature_size=300, band_length=2)

mylsh._build_vocab()
mylsh.encode_binary(to_numpy=True)

mylsh.vectors

mylsh.vectors.shape[1] # this should not be 0
mylsh.vectors.shape
# mylsh.make_signature()
mylsh.cluster()
mylsh.candidates




[{0, 1}, {0, 1}]